### Imports

In [8]:
import random
import numpy as np
from cryptography.hazmat.primitives import hashes

### Problema - KEM (IND-CPA)

Na rosulçao do problema será criada uma classe que implementerá o algoritmo KEM, com o objetivo de ofuscar chaves, que a prórpia classe gera. Para esse cumprir esse objetivo, são necessária implementar 3 funcionalidades:

**Encapsulamento:** Esta função tem o objetivo de ofuscar a chave gerada pelo algoritmo. Para isso, inicialmente, a função gera pequenos erros a partir da função `errorGen`  que gera dois polinómios de tamanho `r`, sendo que a soma dos pesos dos dois erros seja igual a `t`. Após este primeiro passo, gera-se a hash da informação a ser encapsulada utilizando os erros gerados. Para encapsular a informação, é necessário gerar um `r` aleatório denso, utilizando o anel cíclico polinomial R, que será utilizando juntamento com os valores da chave pública e os erros para gerar o encapsulamento da informação k, tal como está na seguinte expressão $(y0,y1)←(r * f0 + e0,r * f1 + e1)$, com $(f0,f1)$ - chave publica e $(e0,e1)$ - pequenos erros.  

**Desencapsulamento:** Para além do encapsulamento, é necessário o desencapsulamento da chave gerada pela classe. Nesta função é calculada a matriz dispersa H e ao syndrome s, sendo estes utilziados no algoritimo `bifFlip`, para descodificar os erros gerados no encapsulamento. Esses erros são necessários para gerar a hash da informação. Em baixo mostram-se os passos necessários para a implementação do desencapsulamento:

* **Geração da matriz dispersa H:** Utilizando a chave privada $(h0,h1)$, a matriz é criada a partir do par de matrizes cíclicas $rot(h0), rot(h1)$, que são calculadas utilizando a função `Rot`. Esta função tem como objetivo gerar a matriz de rotação a partir de um vetor utilizando as funções `toVectorR` e `rot`. Desta forma conseguimos obter a matriz dispensa H = ($rot(h0)|rot(h1)$).

* **Geração do syndrome *s*:** Para calcular o *s* começamos por transformar o encapsulamento, isto é, o criptograma $(y0,y1)$ calculado no *encaps*, num vetor de tamanho *n* utilizando a função `toVectorN`. De seguida utilizamos a expressão $\\,s \\equiv h0\\ast y0 + h1\\ast y1\\,$ para determinar o valor do syndrome *s*.

    * Geramos o novo vetor $x$ igual a *y* e o novo syndrome $z$ igual a *s*, que serão alterados ao longo das iterações.

    * Definimos o número de interações do ciclo que serão iguais ao parâmetro $n$. Caso o s não tenha convergido para 0 ao fim de n interações então ocorreu um problema no desencapsulamento.
    * Enquanto não tivermos atingido o limite de iterações e enquanto o peso de s for diferente de 0:
        * Calculamos o peso dos vários elementos de $z\\cap hj$ com $j\\in\\{1..n\\}$, utilizando a função `hammingWeight`.
        * Calculamos qual o peso máximo desses elementos.
        * Caso o peso de um elemento seja o máximo então vamos alterar os bits da variavél $x$ e atualizar o valor da syndrome $z$ utilizando respetivamente $x_j\\gets \\neg\\;x_j\\;$ e $z\\; \\gets z + h_j$
        * No final, caso o algoritmo convirja então é retornado o valor do $x = (x0,x1)$, caso contrário é retornado o valor NONE pois as iterações atigiram o limite sem o algoritmo ter convergido.
    * **Desencapsulamento da chave:** Para desencapsular a informação é necessário calcular os valores reais de e0 e e1 a partir dos seguintes calculos:
        * Sabendo que $x0 = r * f0 $  e $x1 = r * f1$ então temos:
        $$
            y0 = r * f0 + e0 \\equiv e0 = y0 - r * f0 \\equiv e0 = y0 - x0 \\\\
            y1 = r * f1 + e1 \\equiv e1 = y1 - r * f1 \\equiv e1 = y1 - x1 \\\\
        $$
        * Com estas equações conseguimos obter os valores e0 e e1 que serão usados de forma a verificar a condição $|e0 + e1| = t$, com $|e0 + e1|$ igual à soma dos pessos de e0 e e1. Caso contrário ocorreu um error no processo de desencapsulamento.
        * Finalmente, os valores e0 e e1 serão utilizados para calcular a hash da informação encapsulada e assim desencapsular essa informação.
        
* **Algoritmo Bit Flip:** Este algoritmo iterativo foi implementado na função `bitFlip` e permite alterar os bits $y$ (com *y* a corresponder ao vetor de tamanho *n* que representa o criptograma $(y0,y1)$), atualizando o valor do syndrome $\\,s\\,$ emk cada iteraçõa até que no final a única solução possível da equação  $\\,s\\,=\\, \\sum_{y_j\\neq 0}\\,s\\cap h_j$ que corresponde à definição do *s* é $\\,s = 0\\,$. Utilizando como o input a matriz H, o vetor y e o syndrome s conseguimos implementar o algoritmo através dos seguintes passos:

**Geração do par de chaves:** Esta função tem o objetivo de gerar um par de chaves que será utilizado no encapsulamento  de desencapsulamento da chave gerada pela classe. A função começa por a chave privada do problema a partir da função `coeffGen` que gera dois parâmetros pertencentes a `R` de tamanho `r` cada um com um peso igual a sparse, isto é, com o sparse coeficientes iguais a 1. De seguida gera-se a chave privada segundo a expressão, sendo h0 e h1 os parâmteros da chave privada.

In [ ]:
class BIKE:
    def __init__(self):
        # (block length): a prime number such that 2 is primitive modulo r
        r = 257
        # (error weight): a positive integer
        t = 16
        n = 2*r
        
        # Grupo Finito com 2 elementos
        F2 = GF(2)
        
        # Anel de polinomíos
        F = PolynomialRing(F2, name='w')
        w = F.gen()
        
        # Anel cíclico polinomial F2[X]/<X^r + 1>
        R = QuotientRing(F, F.ideal(w^r + 1))

        self.r = r
        self.t = t
        self.n = n
        
        self.F2 = F2

        self.R = R

    # Gera os coeficentes de um polinómio com tamanho r - utilizado na geração da chave privada e pública
    def coeffGen(self, sparse=3):
        
        coeffs = [1]*sparse + [0]*(self.r-2-sparse)
        random.shuffle(coeffs)
        
        return self.R([1]+coeffs+[1])
    
    # Gera um dois polinomios aleatórios de tamanho r - utilizado para a geração dos erros
    def errorGen(self, t):
        
        res = [1]*t + [0]*(self.n-t)
        random.shuffle(res)
        
        return self.R(res[:self.r]), self.R(res[self.r:])
    
    # Geração do hash - chave encapsulada
    def hashGen(self,e0,e1):
        
        digest = hashes.Hash(hashes.SHA256())
        digest.update(e0.encode())
        digest.update(e1.encode())
        
        return digest.finalize()
    
    # Transformação de um polinómio num vetor de tamanho r
    def toVectorR(self,h):
        
        V = VectorSpace(self.F2, self.r)

        return V(h.list() + [0]*(self.r - len(h.list())))
    
    # Transformação de um par num vetor de tamanho n
    def toVectorN(self, c):
        
        V = VectorSpace(self.F2,self.n)
        
        f = self.toVectorR(c[0]).list() + self.toVectorR(c[1]).list()
        
        return V(f)
    
    # Rotação de uma unidade num vetor 
    def rot(self,m):
        
        V = VectorSpace(self.F2,self.r)
        v = V()
        v[0] = m[-1]
        
        for i in range(self.r-1):
            v[i+1] = m[i]
            
        return v
    
    # Gera matriz de rotação partir de um vetor
    def Rot(self,h):
        
        M = Matrix(self.F2, self.r, self.r)

        M[0] = self.toVectorR(h)
        
        for i in range(1,self.r):
            M[i] = self.rot(M[i-1])
        
        return M
    
    # Gera o peso de hamming de um polinómio binário x
    def hammingWeight(self,x):
        
        return sum([1 if a == 1 else 0 for a in x])
    
    # Implementação do algoritmo de Bit Flip
    def bitFlip(self, H, y, s):
        
        x = y
        z = s
        nIter = self.r
        
        while self.hammingWeight(z) > 0 and nIter > 0:
            
            nIter = nIter - 1
            
            # Todos os pesos de hamming
            weights = [self.hammingWeight(z.pairwise_product(H[i])) for i in range(self.n)]
            maximum = max(weights)
            
            for j in range(self.n):
                
                if weights[j] == maximum:
                    
                    x[j] = 1-x[j]
                    z += H[j]

        if nIter == 0:
            return None
        
        return x
                    
    # Gera um par de chaves - pública e privada
    def keyGen(self):
        
        # Obtenção da chave privada
        h0 = self.coeffGen()
        h1 = self.coeffGen()
        
        # Obtenção da chave pública 
        f = (1, h0/h1)
        
        return (h0,h1), f

In [10]:
class BIKE_KEM(BIKE): 
    def __init__(self):
       BIKE.__init__(self)
    
    # Encapsula uma chave - abordagem McEliece para um KEM-CPA
    def encaps(self, public):
        
        # Gera pequenos erros 
        e0,e1 = self.errorGen(self.t)
        
        # Chave encapsulada
        key = self.hashGen(str(e0),str(e1))
        
        # Gerar aleatoriamente um r <- R denso
        r = self.R.random_element()
        
        # Encapsulamento da chave
        (y0,y1) = (r * public[0] + e0, r * public[1] + e1)
        
        return key, (y0,y1)
    
    # Desencapsula a chave - recebe a chave privada e o encapsulamento
    def decaps(self, private, c):
        
        # Calcula matriz H = rot(h0)|rot(h1)
        h0Rot = self.Rot(private[0])
        h1Rot = self.Rot(private[1])
        H = block_matrix(2,1,[h0Rot,h1Rot])
        
        # Transforma o criptograma c num vetor de tamanho n
        vectorC = self.toVectorN(c)
    
        # Computa syndrome
        s = vectorC * H
        
        # Descodifica s para recuperar o par de erros (e0',e1') utilizando o algoritmo de bitFlip
        error = self.bitFlip(H, vectorC, s)
        
        if error == None:
            print("Iterações atingiram o limite")
            return None
        else:
            
            listError = error.list()
            
            # Erros como par de polinómios
            error0 = self.R(listError[:self.r])
            error1 = self.R(listError[self.r:])
            
            # Como forma de recuperar os erros e0 e e1 originais
            e0 = c[0] - error0
            e1 = c[1] - error1
            
            # Verifica se houve erro no encoding
            if self.hammingWeight(self.toVectorR(e0)) + self.hammingWeight(self.toVectorR(e1)) != self.t:
                
                print("Erro no decoding")
                return None
            else:
                
                # Desencapsula chave 
                key = self.hashGen(str(e0),str(e1))

        return key

In [11]:
bike = BIKE_KEM()

private, public = bike.keyGen()

toEncap, cipheredKey = bike.encaps(public)
print("Original Key: ", toEncap)

toDecap = bike.decaps(private,cipheredKey)
print("Key: ", toDecap)

if toDecap != None and toDecap == toEncap:
    
    print("A chave desencapsulada é igual à original")

Original Key:  b'\x81\xa4z\xd0\r\xad\x17(\x1aS\x9ae[[\xc1\xd9\xdf\x9a<m\x02\xcc\xe4\x19\xeb\x1d\xd5\xc8\r\r\xdbZ'
Key:  b'\x81\xa4z\xd0\r\xad\x17(\x1aS\x9ae[[\xc1\xd9\xdf\x9a<m\x02\xcc\xe4\x19\xeb\x1d\xd5\xc8\r\r\xdbZ'
A chave desencapsulada é igual à original


In [22]:
class BIKE_PKE(BIKE):
    
    def __init__(self):
       BIKE.__init__(self)
    
    # Hash - função g
    def g(self, r):
        
        digest = hashes.Hash(hashes.SHA256())
        digest.update(str(r).encode())
        g = digest.finalize()
        
        return g
    
    # Operação de XOR.
    def xor(self, data, mask):
        
        result = b''
        lengthData = len(data)
        lengthMask = len(mask)
        
        i=0
        
        while i < lengthData:
            
            for j in range(lengthMask):
                
                if i<lengthData:
                    
                    result += (data[i]^^mask[j]).to_bytes(1, byteorder='big')
                    i += 1 
                    
                else:
                    break
                    
        return result
    
    # Núcleo deterministico f - semelhante ao realizado em KEM
    def f(self, public, m, e0, e1):
        
        w = (m * public[0] + e0, m * public[1] + e1)
        
        key = self.hashGen(str(e0),str(e1))
        
        return (key,w)
    
    # Desencapsula a chave gerada pelo o algoritmo - semelhante ao realizado em KEM
    def decapsKey(self,e0,e1):
        
        if self.hammingWeight(self.toVectorR(e0)) + self.hammingWeight(self.toVectorR(e1)) != self.t:
            
            print("Erro no decoding")
            return None
        
        else:
            
            key = self.hashGen(str(e0),str(e1))

        return key
    
    # Desencapsula os erros - semelhante ao realizado em KEM
    def decapsError(self,private, e):
        
        # Calcula matriz H = rot(h0)|rot(h1)
        h0Rot = self.Rot(private[0])
        h1Rot = self.Rot(private[1])
        H = block_matrix(2,1,[h0Rot,h1Rot])
        
        # Transforma o criptograma num vetor de tamanho n
        vectorE = self.toVectorN(e)
    
        # Computa o syndrome
        s = vectorE * H
        
        # Descodifica s para recuperar o vetor (e0,e1)
        error = self.bitFlip(H, vectorE, s)
        
        if error == None:
            print("Iterações atingiram o limite")
            return None
        else:
            
            listError = error.list()
            
            error0 = self.R(listError[:self.r])
            error1 = self.R(listError[self.r:])
            
            e0 = e[0] - error0
            e1 = e[1] - error1
            
        return e0,e1
    
    # Cifra uma mensagem utilizando o FOT - ϑr←h . ϑy←x⊕g(r) . (e,k)←f(y∥r) . ϑc←k⊕r . (y,e,c)
    def encrypt(self, msg, public):
        
        # Gerar erros pequenos
        e0,e1 = self.errorGen(self.t)
        
        # Gerar aleatoriamente r <- R denso - ϑr ← h
        r = self.R.random_element()
        
        # Gerar g(r)
        g = self.g(r)
        
        # Aplicar ϑy ← x ⊕ g(r)
        y = self.xor(msg.encode(),g)
        
        # Transformar a string y num número para ser utilizada pelo o anel R
        yBinary = bin(int.from_bytes(y, byteorder=sys.byteorder))
        ryBinary = self.R(yBinary)
        
        # Aplicar (e,k) ← f(y∥r)
        (key, e) = self.f(public, ryBinary + r, e0, e1)
        
        # Aplicar ϑc ← k ⊕ r
        c = self.xor(str(r).encode(),key)
        
        return y, e, c 
    
    # Decifra uma mensagem utilizando o FOT - ϑk←KREv(e) ⋅ ϑr←c⊕k ⋅ if(e,k)≠f(y∥r) then ⊥  else y⊕g(r)
    def decrypt(self, private,public, y, e, c):
        
        #Aplicar ϑk ← KREv(e)
        e0, e1 = self.decapsError(private,e)
        k = self.decapsKey(e0,e1)
        
        #Aplicar ϑr ← c ⊕ k
        rXOR = self.xor(c,k)
        r = self.R(rXOR.decode())
        
        #Aplicar as mesmas transformações associadas ao processor de cifra
        yBinary = bin(int.from_bytes(y, byteorder=sys.byteorder))
        ryBinary = self.R(yBinary)
        
        #Aplicar if(e,k)≠f(y∥r) then ⊥  else y⊕g(r)
        if (k,e) != self.f(public, ryBinary + r, e0, e1):
            
            print("Erro no decoding")
            return None
        
        else:
            
            #Gerar g(r)
            g = self.g(r)
            
            #Aplicar y ⊕ g(r)
            plaintext = self.xor(y,g)
            
        return plaintext

In [29]:
bikePKE = BIKE_PKE()
msg = "Mensagem a ser cifrada"
print("Mensagem original: " + msg)

private, public = bikePKE.keyGen()

msgEncaps, keyEncaps, ciphertext = bikePKE.encrypt(msg, public)
print("Ciphertext: ")
print(ciphertext)

plaintext = bikePKE.decrypt(private, public, msgEncaps, keyEncaps, ciphertext)
print("Plaintext: " + plaintext.decode())

Mensagem original: Mensagem a ser cifrada
Ciphertext: 
b'\x9b.\xfc\xf2\xc6\x87\x04b\xac#)\xb7\xce>\xa4\x9d?0=%\xae.\xd5\xaa\xa3_\xd7\x17\xa1\x0b\x90\xe0\xcc;\xff\xe1\xea\xeb\x03`\xb5("\xe0\xdb=\xb7\xb1S78=\xa5%\x82\xbf\xa0L\xfb{\xa6\x0f\x87\xeb\xc7l\xea\xe2\xf9\xc7of\xb8>)\xeb\x8c(\xb4\xa2\x7f[>1\xb0.\x89\xe8\xb5O\xe8W\xca\t\x84\xff\xccg\xbd\xf7\xfa\xd4C\n\xbe;0\xe0\x87\x7f\xa1\xa1lwR7\xb68\x82\xe3\xe2Z\xebD\xe6e\x82\xf8\xd8l\xb6\xa0\xef\xd7P&\xd2::\xf0\x8ct\xf6\xb4od~[\xb7<\x9b\xe8\xe9\r\xfeG\xf5I\xee\xf9\xdet\xbd\xab\xb8\xc2S5\xfeV;\xf2\x99\x7f\xfd\xe3zgmw\xdb<\x90\xf9\xe2\x06\xa9R\xf6Z\xc2\x95\xde~\xad\xa0\xb3\x95F6\xedzW\xf2\x9df\xf6\xe8-rnd\xf7P\x90\xf9\xf4\r\xa2\x05\xe3Y\xd1\xb9\xb2~\xac\xb3\xb8\x9e\x11#\xeei{\x9e\x9eo\xef\xe3&%{g\xe4|\xfc\xfa\xf2\x1a\xa9\x0e\xb4L\xd2\xaa\x9e\x12\xaf\xb0\xae\x95\x1at\xfbjh\xb2\xf2m\xe6\xf6-.,r\xe7o\xd0\x96\xf3\x14\xbc\x05\xbf\x1b\xc7\xa9\x8d>\xc3\xb1\xa1\x86\x11\x7f\xac\x7fk\xa1\xde\x01\xe7\xfb4%\'%\xf2l\xc3\xba\x9c\x1c\xb1\x12\xb4\x10\x90\xbc\x8